In [1]:
import matplotlib.pyplot as plt

from src.utils import *
import os
import IPython.display as ipd
import logging
from src.ResNet1D.resnet1d_model import SpectrogramModel1D
from src.resnet_model import SpectrogramModel
import torch.nn as nn
import librosa


logging.getLogger('numba').setLevel(logging.WARNING)
logging.getLogger('matplotlib.font_manager').disabled = True
logging.getLogger('matplotlib.colorbar').disabled = True
logging.getLogger('matplotlib.pyplot').disabled = True

In [2]:
config_path = '../config/residualnet_train_config.yaml'
config_res = read_yaml(config_path)
seed_everything(1234)
set_gpu(-1)
plt.style.use('dark_background')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
script_dir = os.getcwd()

GPU selected: 0 - NVIDIA GeForce RTX 3060


In [3]:
# load one file
file_number = 8844552
label = np.array([1])
clean_audio_path = os.path.join(script_dir, '..',f'/nas/public/dataset/asvspoof2019/LA/ASVspoof2019_LA_eval/flac/LA_E_{file_number}.flac')
pert_audio_path = os.path.join(script_dir,f'Ensemble1D/QUANT_ENS1D_v0_10_10_3s_None/EnsembleBIM_10_10_ResRaw_v0_pow_LA_E_{file_number}_None.flac')
clean, _ = librosa.load(clean_audio_path, sr=16000)
pert, _ = librosa.load(pert_audio_path, sr=16000)
clean = clean[:47104]
pert = pert[:47104]

In [4]:
clean.shape

(47104,)

In [5]:
ipd.Audio(pert, rate=16000)

In [6]:
# load model
parent_dir = os.path.dirname(script_dir)
config_path = os.path.join(parent_dir, 'config', 'residualnet_train_config.yaml')
config = read_yaml(config_path)
check_dir = os.path.join(parent_dir, config['model_path_spec_pow_v0'])
model = SpectrogramModel().to(device)
model.load_state_dict(torch.load(check_dir, map_location=device), strict=False)

<All keys matched successfully>

In [7]:
# # load model
# 
# parent_dir = os.path.dirname(script_dir)
# config_path = os.path.join(parent_dir, 'config/resnet1d.yaml')
# config = read_yaml(config_path)
# 
# check_dir = os.path.join(parent_dir, config['model_path_spec_pow_v0'])
# 
# model = SpectrogramModel1D().to(device)
# model.load_state_dict(torch.load(check_dir, map_location=device), strict=False)
# model.eval()

In [8]:
win_length = 2048
n_fft = 2048
hop_length = 512
window = 'hann'

s = librosa.stft(clean, n_fft=n_fft, win_length=win_length, hop_length=hop_length, window=window, center=True)
phase = np.angle(s)

a = np.abs(s) ** 2
pow_spec_c = librosa.power_to_db(a, ref=np.max)

In [9]:
win_length = 2048
n_fft = 2048
hop_length = 512
window = 'hann'

s = librosa.stft(pert, n_fft=n_fft, win_length=win_length, hop_length=hop_length, window=window, center=True)
phase = np.angle(s)

a = np.abs(s) ** 2
pow_spec_p = librosa.power_to_db(a, ref=np.max)

In [10]:
# clean mini batch
batch_x = torch.from_numpy(pow_spec_c).unsqueeze(dim=0).to(device)
# pert mini batch
batch_x_p = torch.from_numpy(pow_spec_p).unsqueeze(dim=0).to(device)

In [11]:
batch_x.shape

torch.Size([1, 1025, 93])

In [12]:
batch_x.requires_grad = True
out = model(batch_x)

In [13]:
out_ = model(batch_x_p)

In [14]:
torch.argmax(out)

tensor(1, device='cuda:0')

In [15]:
torch.argmax(out_)

tensor(0, device='cuda:0')